# **NBA ETL Pipeline**

This notebook builds a basic ETL (Extract, Transform, Load) pipeline using the `nba_api` to collect and store NBA team and player data for the 2023–24 season. Data is saved to Google Drive for reuse and analysis.

### **Environment Setup**

In [16]:
# Install nba_api if not already installed
!pip install nba_api

# Import necessary libraries
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import leaguegamefinder, playercareerstats, leaguedashteamstats, leaguedashplayerstats, leaguestandings, commonteamroster
from google.colab import drive
import os
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Team Stats: Extract**

In [17]:
# Pull basic team stats for the 2023-24 season
team_stats = leaguedashteamstats.LeagueDashTeamStats(
    season='2023-24',
    measure_type_detailed_defense='Base',
    per_mode_detailed='PerGame'
)

# Convert to DataFrame
team_stats_df = team_stats.get_data_frames()[0]

## **Team Stats: Transform**

In [21]:
# Check for column existence using correct casing
cols_to_drop = ['TEAM_ID']  # Match the actual column name
existing_cols = [col for col in cols_to_drop if col in team_stats_df.columns]

# Drop only the existing columns
team_stats_cleaned = team_stats_df.drop(columns=existing_cols)

# Then clean column names (after dropping)
team_stats_cleaned.columns = team_stats_cleaned.columns.str.lower().str.replace(' ', '_')

# Preview
team_stats_cleaned.head()

,team_name,gp,w,l,w_pct,min,fgm,fga,fg_pct,fg3m,...,reb_rank,ast_rank,tov_rank,stl_rank,blk_rank,blka_rank,pf_rank,pfd_rank,pts_rank,plus_minus_rank
0,Atlanta Hawks,82,36,46,0.439,48.4,43.0,92.5,0.465,13.7,...,6,16,16,16,26,22,15,7,5,21
1,Boston Celtics,82,64,18,0.780,48.4,43.9,90.2,0.487,16.5,...,2,14,1,27,1,1,2,30,2,1
2,Brooklyn Nets,82,32,50,0.390,48.3,40.7,89.1,0.456,13.3,...,11,20,12,28,12,16,13,21,25,22
3,Charlotte Hornets,82,21,61,0.256,48.1,40.0,87.0,0.460,12.1,...,30,26,18,25,25,13,7,29,28,30
4,Chicago Bulls,82,39,43,0.476,48.7,42.0,89.5,0.470,11.5,...,14,23,3,9,18,14,17,14,22,20


## **Team Stats: Load**

In [22]:
# Define target directory
output_dir = '/content/drive/My Drive/MTurner Portfolio/NBA ETL Pipeline/data files'

# Create the folder if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save the cleaned DataFrame to CSV
file_path = os.path.join(output_dir, 'team_stats_2023_24.csv')
team_stats_cleaned.to_csv(file_path, index=False)

print(f"File saved to: {file_path}")

File saved to: /content/drive/My Drive/MTurner Portfolio/NBA ETL Pipeline/data files/team_stats_2023_24.csv


## **Player Stats: Extract**

In [30]:
# Search for player IDs
tatum_id = players.find_players_by_full_name("Jayson Tatum")[0]['id']
brown_id = players.find_players_by_full_name("Jaylen Brown")[0]['id']

# Get career stats
tatum_career = playercareerstats.PlayerCareerStats(player_id=tatum_id)
brown_career = playercareerstats.PlayerCareerStats(player_id=brown_id)

# Convert to DataFrames
tatum_df = tatum_career.get_data_frames()[0]
brown_df = brown_career.get_data_frames()[0]

## **Player Stats: Transform**

In [24]:
# Clean column names
tatum_2024.columns = tatum_2024.columns.str.lower().str.replace(' ', '_')
brown_2024.columns = brown_2024.columns.str.lower().str.replace(' ', '_')

## **Player Stats: Load**

In [31]:
# Save with new filenames for clarity
tatum_df.to_csv(os.path.join(output_dir, 'jayson_tatum_stats.csv'), index=False)
brown_df.to_csv(os.path.join(output_dir, 'jaylen_brown_stats.csv'), index=False)

print("Full career stats saved!")

Full career stats saved!


## **Game Logs: Extract**

In [79]:
# Celtics team ID
celtics_id = 1610612738

# Pull full game logs
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
games_df = gamefinder.get_data_frames()[0]

## **Game Logs: Transform**

In [82]:
# Clean column names
games_df.columns = games_df.columns.str.lower().str.replace(' ', '_')

# Filter to 2023–24 regular season only
games_df = games_df[games_df['season_id'] == '22024'].copy()

# Add season_phase column
def classify_season_phase(season_id):
    if season_id.startswith("1"):
        return "preseason"
    elif season_id.startswith("2"):
        return "regular"
    elif season_id.startswith("3"):
        return "play-in"
    elif season_id.startswith("4"):
        return "playoffs"
    elif season_id.startswith("5"):
        return "all-star"
    else:
        return "unknown"

games_df.loc[:, 'season_phase'] = games_df['season_id'].astype(str).apply(classify_season_phase)

## **Game Logs: Load**

In [83]:
games_df.to_csv(os.path.join(output_dir, 'celtics_game_logs_2023_24.csv'), index=False)
print("2023–24 regular season game logs saved.")

2023–24 regular season game logs saved.


## **League Standings: Extract**

In [37]:
standings = leaguestandings.LeagueStandings(season='2023-24')
standings_df = standings.get_data_frames()[0]

## **League Standings: Transform**

In [38]:
# Clean and filter for East only
standings_df.columns = standings_df.columns.str.lower().str.replace(' ', '_')
east_df = standings_df[standings_df['conference'] == 'East']

## **League Standings: Load**

In [39]:
east_df.to_csv(os.path.join(output_dir, 'eastern_conference_standings_2023_24.csv'), index=False)
print("Eastern Conference standings saved!")

Eastern Conference standings saved!


## **Team Roster: Extract**

In [40]:
roster = commonteamroster.CommonTeamRoster(
    team_id=celtics_id,
    season='2023-24'
)
roster_df = roster.get_data_frames()[0]

## **Team Roster: Transform**

In [41]:
roster_df.columns = roster_df.columns.str.lower().str.replace(' ', '_')

## **Team Roster: Load**

In [42]:
roster_df.to_csv(os.path.join(output_dir, 'celtics_roster_2023_24.csv'), index=False)
print("Celtics roster saved.")

Celtics roster saved.


## **ETL Summary & Samples**

In [69]:
# Load and preview 5 rows from each saved file
team_sample = pd.read_csv(os.path.join(output_dir, 'team_stats_2023_24.csv')).head()
player_sample = pd.read_csv(os.path.join(output_dir, 'jaylen_brown_stats.csv')).head()
game_sample = pd.read_csv(os.path.join(output_dir, 'celtics_game_logs_2023_24.csv')).head()
standings_sample = pd.read_csv(os.path.join(output_dir, 'eastern_conference_standings_2023_24.csv')).head()
roster_sample = pd.read_csv(os.path.join(output_dir, 'celtics_roster_2023_24.csv')).head()

# Display dataframes
print("Team Stats Sample:")
display(team_sample)

print("Jaylen Brown Career Stats Sample:")
display(player_sample)

print("Celtics Game Logs Sample:")
display(game_sample)

print("Eastern Conference Standings Sample:")
display(standings_sample)

print("Celtics Roster Sample:")
display(roster_sample)

Team Stats Sample:


,team_name,gp,w,l,w_pct,min,fgm,fga,fg_pct,fg3m,...,reb_rank,ast_rank,tov_rank,stl_rank,blk_rank,blka_rank,pf_rank,pfd_rank,pts_rank,plus_minus_rank
0,Atlanta Hawks,82,36,46,0.439,48.4,43.0,92.5,0.465,13.7,...,6,16,16,16,26,22,15,7,5,21
1,Boston Celtics,82,64,18,0.780,48.4,43.9,90.2,0.487,16.5,...,2,14,1,27,1,1,2,30,2,1
2,Brooklyn Nets,82,32,50,0.390,48.3,40.7,89.1,0.456,13.3,...,11,20,12,28,12,16,13,21,25,22
3,Charlotte Hornets,82,21,61,0.256,48.1,40.0,87.0,0.460,12.1,...,30,26,18,25,25,13,7,29,28,30
4,Chicago Bulls,82,39,43,0.476,48.7,42.0,89.5,0.470,11.5,...,14,23,3,9,18,14,17,14,22,20


Jaylen Brown Career Stats Sample:


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1627759,2016-17,0,1610612738,BOS,20.0,78,20,1341.0,192,...,0.685,45,175,220,64,35,18,68,142,515
1,1627759,2017-18,0,1610612738,BOS,21.0,70,70,2152.0,373,...,0.644,66,280,346,114,70,26,124,181,1017
2,1627759,2018-19,0,1610612738,BOS,22.0,74,25,1913.0,368,...,0.658,65,248,313,100,69,32,99,186,964
3,1627759,2019-20,0,1610612738,BOS,23.0,57,57,1934.0,427,...,0.724,62,302,364,118,65,20,125,167,1159
4,1627759,2020-21,0,1610612738,BOS,24.0,58,58,1999.0,538,...,0.764,71,276,347,195,72,32,158,168,1430


Celtics Game Logs Sample:


,season_id,team_id,team_abbreviation,team_name,game_id,game_date,matchup,wl,min,pts,...,oreb,dreb,reb,ast,stl,blk,tov,pf,plus_minus,season_phase


Eastern Conference Standings Sample:


,leagueid,seasonid,teamid,teamcity,teamname,conference,conferencerecord,playoffrank,clinchindicator,division,...,may,jun,jul,aug,sep,oct,nov,dec,preas,postas
0,0,22023,1610612738,Boston,Celtics,East,41-11,1,- e,Atlantic,...,NaN,NaN,NaN,NaN,NaN,3-0,11-4,12-2,43-12,21-6
1,0,22023,1610612752,New York,Knicks,East,35-17,2,- x,Atlantic,...,NaN,NaN,NaN,NaN,NaN,2-2,9-5,6-8,33-22,17-10
2,0,22023,1610612749,Milwaukee,Bucks,East,34-18,3,- c,Central,...,NaN,NaN,NaN,NaN,NaN,2-1,11-5,11-2,35-21,14-12
3,0,22023,1610612739,Cleveland,Cavaliers,East,31-21,4,- x,Central,...,NaN,NaN,NaN,NaN,NaN,1-3,9-6,8-5,36-17,12-17
4,0,22023,1610612753,Orlando,Magic,East,32-20,5,- se,Southeast,...,NaN,NaN,NaN,NaN,NaN,2-2,11-3,6-8,30-25,17-10


Celtics Roster Sample:


,teamid,season,leagueid,player,nickname,player_slug,num,position,height,weight,birth_date,age,exp,school,player_id,how_acquired
0,1610612738,2023,0,Jayson Tatum,Jayson,jayson-tatum,0,F-G,6-8,210,"MAR 03, 1998",26.0,6,Duke,1628369,#3 Pick in 2017 Draft
1,1610612738,2023,0,Jrue Holiday,Jrue,jrue-holiday,4,G,6-4,205,"JUN 12, 1990",34.0,14,UCLA,201950,Traded from POR on 10/01/23
2,1610612738,2023,0,Jaylen Brown,Jaylen,jaylen-brown,7,G-F,6-6,223,"OCT 24, 1996",27.0,7,California,1627759,#3 Pick in 2016 Draft
3,1610612738,2023,0,Kristaps Porziņģis,Kristaps,kristaps-porziņģis,8,F-C,7-2,240,"AUG 02, 1995",28.0,7,Cajasol Sevilla,204001,Traded from WAS on 06/23/23
4,1610612738,2023,0,Derrick White,Derrick,derrick-white,9,G,6-4,190,"JUL 02, 1994",29.0,6,Colorado,1628401,Traded from SAS on 02/10/22
